# CARGA DEL DATASET Y LIBRERIAS

## Importacion de Librerias

In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Peter\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

## Carga del dataset crudo para su tratamiento

In [3]:
data = []

# Leer el archivo línea por línea y cargar los objetos JSON
#with open('../raw_data/australian_user_reviews.json', encoding='utf-8') as json_file:
with open(f'D:\PEDRO\AHORA\A LA CARGA!!\WORKING AREA\Cursos and Tuts\Henry\Labs\PIML3\DESARROLLO DEL PI\A Local/raw_data/australian_user_reviews.json', encoding='utf-8') as json_file:
    for line in json_file:
        data.append(eval(line))  # Convertir la línea en un objeto Python

# Crear el DataFrame a partir de la lista de objetos JSON
df_user_reviews = pd.DataFrame(data)

# LIMPIEZA Y TRANSFORMACION DE DATASET CRUDO

## Exploracion Basica

In [ ]:
print('Numero de filas y campos: ',df_user_reviews.shape)
print('Tipos de las columnas:')
print(df_user_reviews.dtypes)
print('Datos faltantes:')
print(df_user_reviews.isnull().sum())
print("Primera fila de los campos 'user_id' 'user_url':")
df_user_reviews.head(1)

#df_user_reviews_userdata_prepared[['user_id','user_url']].head(5)

Numero de filas y campos:  (25799, 3)
Tipos de las columnas:
user_id     object
user_url    object
reviews     object
dtype: object
Datos faltantes:
user_id     0
user_url    0
reviews     0
dtype: int64
Primera fila de los campos 'user_id' 'user_url':


,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."


Al parecer el campo 'reviews' esta anidado; checkamos su primer valor

In [ ]:
print("Primer contenido de las columna 'reviews':")
df_user_reviews['reviews'][0]

Primer contenido de las columna 'reviews':


[{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': 'Posted July 15, 2011.',
  'last_edited': '',
  'item_id': '22200',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."},
 {'funny': '',
  'posted': 'Posted April 21, 2011.',
  'last_edited': '',
  'item_id': '43110',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]

Vemos que debemos desanidar 'reviews', pero previo a eso existen tratamientos previos que realizar

## Tratamientos previos al desanidado del campo 'reviews'

Antes del desanidado verificamos si existen ides duplicados en 'user_id'. Para eliminar estos duplicados primero debemos verificar registros con valores faltantes en 'reviews', pues de nada sirven registros con ides unicos en 'user_id' y valores vacios en 'reviews'

### Exploramos el campo 'user_id'

Los ides deben ser unicos, por lo que verificamos si existen valores repetidos

In [ ]:
#Muestra los valores unicos en 'user_id'
ides = df_user_reviews['user_id'] #muestra los valores en 'user_id'
print("Total de valores en 'user_id': ", len(ides))

ides_unicos = df_user_reviews['user_id'].unique() #muestra valores únicos en  'user_id'
print("Número de valores únicos en 'user_id': ", len(ides_unicos))

Total de valores en 'user_id':  25799
Número de valores únicos en 'user_id':  25485


Encontramos muchos valores repetidos en 'user_id'      
Para analizar estas ocurrencias vamos a capturar 3 ides que se repiten y mostrar todas las filas implicadas  
Para acelerar la tarea trabajamos sin el campo 'reviews'

In [ ]:
# Filtrar las columnas a mantener
columnas_a_mantener = ['user_id','user_url']
df_user_reviews_no_reviews = df_user_reviews[columnas_a_mantener]

# Identifica todos los user_id duplicados y cuenta sus ocurrencias
duplicados = df_user_reviews_no_reviews['user_id'].value_counts()

# Captura en una lista tres user_id con ocurrencias repetidas .
user_id_repetidos = duplicados[duplicados > 1].head(3).index.tolist()
print('Los 3 primeros ides repetidos son:',user_id_repetidos)

# Filtra y muestra las filas correspondientes a estos user_id
filas_repetidas = df_user_reviews_no_reviews[df_user_reviews_no_reviews['user_id'].isin(user_id_repetidos)]
filas_repetidas

Los 3 primeros ides repetidos son: ['76561198027488037', '76561198045953692', '76561198051777058']


,user_id,user_url
1103,76561198045953692,http://steamcommunity.com/profiles/76561198045...
2197,76561198051777058,http://steamcommunity.com/profiles/76561198051...
5098,76561198045953692,http://steamcommunity.com/profiles/76561198045...
6935,76561198027488037,http://steamcommunity.com/profiles/76561198027...
6936,76561198027488037,http://steamcommunity.com/profiles/76561198027...
7364,76561198051777058,http://steamcommunity.com/profiles/76561198051...
12678,76561198045953692,http://steamcommunity.com/profiles/76561198045...
15693,76561198027488037,http://steamcommunity.com/profiles/76561198027...
16451,76561198051777058,http://steamcommunity.com/profiles/76561198051...


### Tratamiento del Campo 'reviews'

Antes de eliminar ides repetidos checkamos si algo extraño en el campo 'reviews' de esos 3 ides repetidos  
Para esto usamos 'df_user_reviews'

In [ ]:
for id_repetido in user_id_repetidos:
    filas_repetida = df_user_reviews[df_user_reviews['user_id'] == id_repetido]
    print(filas_repetida)

                 user_id                                           user_url   
6935   76561198027488037  http://steamcommunity.com/profiles/76561198027...  \
6936   76561198027488037  http://steamcommunity.com/profiles/76561198027...   
15693  76561198027488037  http://steamcommunity.com/profiles/76561198027...   

                                                 reviews  
6935   [{'funny': '', 'posted': 'Posted May 12.', 'la...  
6936   [{'funny': '', 'posted': 'Posted May 12.', 'la...  
15693  [{'funny': '', 'posted': 'Posted May 12.', 'la...  
                 user_id                                           user_url   
1103   76561198045953692  http://steamcommunity.com/profiles/76561198045...  \
5098   76561198045953692  http://steamcommunity.com/profiles/76561198045...   
12678  76561198045953692  http://steamcommunity.com/profiles/76561198045...   

                                                 reviews  
1103   [{'funny': '', 'posted': 'Posted July 23, 2014...  
5098   [{'fu

Vemos que efectivamente existen registros repetidos en 'df_user_reviews', pero en cuanto al campo 'reviews', solo checkando 3 registros no es posible sacar conclusiones.  
Intentamos eliminar datos faltantes en 'reviews'

In [ ]:
print('Numero de filas y campos antes de la eliminacion: ',df_user_reviews.shape)
df_user_reviews = df_user_reviews[df_user_reviews['reviews'].apply(lambda x: len(x) > 0)]
print('Numero de filas y campos despues de la eliminacion: ',df_user_reviews.shape)

Numero de filas y campos antes de la eliminacion:  (25799, 3)
Numero de filas y campos despues de la eliminacion:  (25771, 3)


### Tratamiento del Campo 'user_id'

Ahora tratamos las filas con ides repetidos. Como estos deben ser unicos, dejamos solo un ejemplar y eliminamos el resto

In [ ]:
print('Numero de filas y campos antes de la eliminacion: ',df_user_reviews.shape)
df_user_reviews = df_user_reviews.drop_duplicates(subset='user_id', keep='first')
print('Numero de filas y campos despues de la eliminacion: ',df_user_reviews.shape)

Numero de filas y campos antes de la eliminacion:  (25771, 3)
Numero de filas y campos despues de la eliminacion:  (25458, 3)


Verificaciones finales:
* Contando las veces que se repiten los valores de 'user_id'
* Contando los valores únicos en 'user_id'

In [ ]:
ides_unicos = df_user_reviews['user_id'].unique() 
print("Conteo de valores únicos en 'user_id': ", len(ides_unicos))
print('\n')

repeticiones = df_user_reviews['user_id'].value_counts()
print("Conteo de valores repetidos en 'user_id':")
print(repeticiones)

Conteo de valores únicos en 'user_id':  25458


Conteo de valores repetidos en 'user_id':
user_id
76561197970982479    1
76561198060398831    1
76561198049868108    1
Woodyjoseph          1
CurseDFlaPPy         1
                    ..
Teoh1189             1
Fuzz_Melon           1
macintosh1122        1
76561198086687010    1
LydiaMorley          1
Name: count, Length: 25458, dtype: int64


## Desanidando el campo 'reviews'

In [ ]:
print('columnas en df_user_reviews: ','[' + ', '.join(df_user_reviews.keys()) + ']')

columnas en df_user_reviews:  [user_id, user_url, reviews]


In [ ]:
print('Numero de filas y campos antes del desanidado: ',df_user_reviews.shape)
print('columnas antes del desanidado: ','[' + ', '.join(df_user_reviews.keys()) + ']')
print('Datos faltantes antes del desanidado:')
print(df_user_reviews.isnull().sum())
data_desanidada = []

for index, row in df_user_reviews.iterrows():
    user_id = row['user_id']
    user_url = row['user_url']
    reviews_list = row['reviews']

    for item in reviews_list:
        new_row = {
            'user_id': user_id,
            'user_url': user_url,            
            'funny': item.get('funny', ''),
            'posted': item.get('posted', ''),
            'last_edited': item.get('last_edited', ''),
            'item_id': item.get('item_id', ''),
            'helpful': item.get('helpful', ''),
            'recommend': item.get('recommend', ''),
            'review': item.get('review', '')
        }

        data_desanidada.append(new_row)

df_user_reviews = pd.DataFrame(data_desanidada)

print('\n')
print('Numero de filas y campos despues del desanidado: ',df_user_reviews.shape)
print('columnas despues del desanidado: ','[' + ', '.join(df_user_reviews.keys()) + ']')
print('Datos faltantes despues del desanidado:')
print(df_user_reviews.isnull().sum())


Numero de filas y campos antes del desanidado: 

 (25458, 3)
columnas antes del desanidado:  [user_id, user_url, reviews]
Datos faltantes antes del desanidado:
user_id     0
user_url    0
reviews     0
dtype: int64


Numero de filas y campos despues del desanidado:  (58431, 9)
columnas despues del desanidado:  [user_id, user_url, funny, posted, last_edited, item_id, helpful, recommend, review]
Datos faltantes despues del desanidado:
user_id        0
user_url       0
funny          0
posted         0
last_edited    0
item_id        0
helpful        0
recommend      0
review         0
dtype: int64


## Tratamiento del campo 'item_id'

El campo 'item_id' es el campo mas importante del dataset despues del campo 'user_id'.  
Cada usuario debe haber reseñado sus items una sola vez, por lo que nos aseguramos que para cada 'user_id' no haya valores repetidos del campo 'item_id'.

### Exploramos 'item_id'

In [ ]:
print('columnas en df_user_reviews: ','[' + ', '.join(df_user_reviews.keys()) + ']')

columnas en df_user_reviews:  [user_id, user_url, funny, posted, last_edited, item_id, helpful, recommend, review]


In [ ]:
print(df_user_reviews.shape)
print("Tipo de dato en 'item_id':",df_user_reviews['item_id'].dtype)
df_user_reviews.head(3)

(58431, 9)
Tipo de dato en 'item_id': object


,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...


Todos los valores en la columna 'item_id' deben ser digitos, es decir con formato tipo `r'^\d+$'`  
Verificamos si hay valores que no cumplan tal formato

In [ ]:
# Busca valores en 'item_id' que no sigan el formato deseado
valores_no_validos = df_user_reviews[~df_user_reviews['item_id'].astype(str).str.contains(r'^\d+$')]

print("Valores en 'item_id' sin el formato 'r'^\d+$': ", len(valores_no_validos['item_id'])) #muestra la cantidad de valores sin el formato

Valores en 'item_id' sin el formato 'r'^\d+$':  0


Checkamos si hay valores repetidos en 'item_id'. Para eso capturamos tres ides repetidos en 'user_id'

In [ ]:
# Filtrar las columnas a mantener
columnas_a_mantener = ['user_id','item_id']
df_user_reviews_no_full = df_user_reviews[columnas_a_mantener]# Identifica todos los user_id duplicados y cuenta sus ocurrencias
duplicados = df_user_reviews_no_full['user_id'].value_counts()

# Captura en una lista tres user_id con ocurrencias repetidas .
user_id_repetidos = duplicados[duplicados > 1].head(3).index.tolist()

# Filtra y muestra las filas correspondientes a estos user_id
filas_repetidas = df_user_reviews_no_full[df_user_reviews_no_full['user_id'].isin(user_id_repetidos)]

for id_repetido in user_id_repetidos:
    filas_repetida = df_user_reviews_no_full[df_user_reviews['user_id'] == id_repetido]
    print(filas_repetida)

                 user_id item_id
17466  76561198108415635  238910
17467  76561198108415635    6900
17468  76561198108415635  284770
17469  76561198108415635  221380
17470  76561198108415635  265930
17471  76561198108415635  307230
17472  76561198108415635   17390
17473  76561198108415635  301910
17474  76561198108415635  105600
17475  76561198108415635    9480
      user_id item_id
15623  NanoPi     280
15624  NanoPi  261570
15625  NanoPi     550
15626  NanoPi  248310
15627  NanoPi  268750
15628  NanoPi  259080
15629  NanoPi  237930
15630  NanoPi  236090
15631  NanoPi  225260
15632  NanoPi   29180
        user_id item_id
12111  banksyyo  275850
12112  banksyyo  374320
12113  banksyyo  333950
12114  banksyyo  218620
12115  banksyyo  438680
12116  banksyyo   17390
12117  banksyyo  233130
12118  banksyyo  417860
12119  banksyyo  105800
12120  banksyyo   12140


A simple vista no se ven valores repetidos en 'item_id'. Nos aseguramos  
Verificamos si existen 'item_id' repetidos en las filas que tienen el mismo 'user_id'

In [ ]:
duplicados = df_user_reviews[df_user_reviews.duplicated(subset=['user_id', 'item_id'], keep=False)]
print('Cantidad de "item_id" repetidos por cada usuario: ',len(duplicados))

Cantidad de "item_id" repetidos por cada usuario:  0


## Tratamiento del campo 'posted'

### Exploramos 'posted'

In [ ]:
print('columnas en df_user_reviews: ','[' + ', '.join(df_user_reviews.keys()) + ']')

columnas en df_user_reviews:  [user_id, user_url, funny, posted, last_edited, item_id, helpful, recommend, review]


In [ ]:
print(df_user_reviews.shape)
print(df_user_reviews.dtypes)
df_user_reviews.head(3)

(58431, 9)
user_id        object
user_url       object
funny          object
posted         object
last_edited    object
item_id        object
helpful        object
recommend        bool
review         object
dtype: object


,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...


Listamos varias ocurrencias de 'posted'

In [ ]:
posted_muestra = df_user_reviews['posted'].head(20)
matriz = np.array(posted_muestra).reshape(10, 2) # Convierte la serie 'posted_muestra' en un arreglo 15x7
print(matriz)

[['Posted November 5, 2011.' 'Posted July 15, 2011.']
 ['Posted April 21, 2011.' 'Posted June 24, 2014.']
 ['Posted September 8, 2013.' 'Posted November 29, 2013.']
 ['Posted February 3.' 'Posted December 4, 2015.']
 ['Posted November 3, 2014.' 'Posted October 15, 2014.']
 ['Posted October 15, 2014.' 'Posted October 15, 2014.']
 ['Posted October 14, 2013.' 'Posted July 28, 2012.']
 ['Posted June 2, 2012.' 'Posted June 29, 2014.']
 ['Posted November 22, 2012.' 'Posted February 23, 2012.']
 ['Posted April 15, 2014.' 'Posted December 23, 2013.']]


Vemos que los valores de 'posted' no estan en el formato correcto 'YYYY-MM-DD' 

### Transformamos los valores de 'posted'a formato 'YYYY-MM-DD'

Checkamos si hay elementos en 'posted' que no empiezan con 'Posted'

In [ ]:
elementos_no_posted = df_user_reviews[~df_user_reviews['posted'].str.startswith('Posted ')]['posted']

print('Elementos encontrados: ',len(elementos_no_posted))
print(elementos_no_posted)

Elementos encontrados:  0
Series([], Name: posted, dtype: object)


Removemos la palabra Posted, la coma y el punto al final

In [ ]:
df_user_reviews['posted'] = df_user_reviews['posted'].str.replace('Posted ', '').str.replace(',', '').str.rstrip('.')

Listamos varias ocurrencias de 'posted'

In [ ]:
posted_muestra = df_user_reviews['posted'].head(24)
matriz = np.array(posted_muestra).reshape(8, 3) # Convierte la serie 'posted_muestra' en un arreglo 8x3
print(matriz)

[['November 5 2011' 'July 15 2011' 'April 21 2011']
 ['June 24 2014' 'September 8 2013' 'November 29 2013']
 ['February 3' 'December 4 2015' 'November 3 2014']
 ['October 15 2014' 'October 15 2014' 'October 15 2014']
 ['October 14 2013' 'July 28 2012' 'June 2 2012']
 ['June 29 2014' 'November 22 2012' 'February 23 2012']
 ['April 15 2014' 'December 23 2013' 'March 14 2014']
 ['July 11 2013' 'May 5 2014' 'December 24 2012']]


Vemos que en la tercera fila el valor 'February 3' no cumple la apariencia `'<mes> <dia>, <año>'`. Cuantificamos valores similares 

In [ ]:
total = df_user_reviews.shape[0]
cumple_formato = df_user_reviews[df_user_reviews['posted'].str.contains(r'[A-Z][a-z]+ \d{1,2} \d{4}')]
no_cumple_formato = df_user_reviews[~df_user_reviews['posted'].str.contains(r'[A-Z][a-z]+ \d{1,2} \d{4}')]

print(f"De los {total} elementos de 'posted', {len(cumple_formato['posted'])} cumplen el formato '<mes> <dia> <año>' y {len(no_cumple_formato['posted'])} no lo cumplen")

De los 58431 elementos de 'posted', 48498 cumplen el formato '<mes> <dia> <año>' y 9933 no lo cumplen


Eliminamos valores en 'posted' que no cumplen el formato `'<mes> <dia> <año>'`

In [ ]:
print('Numero de filas y campos antes de eliminar: ',df_user_reviews.shape)
df_user_reviews = df_user_reviews[df_user_reviews['posted'].str.contains(r'[A-Z][a-z]+ \d{1,2} \d{4}')]
print('Numero de filas y campos despues de eliminar: ',df_user_reviews.shape)


Numero de filas y campos antes de eliminar:  (58431, 9)
Numero de filas y campos despues de eliminar:  (48498, 9)


Cambiamos el formato tipo `'<mes> <dia> <año>'` a 'YYYY-MM-DD'  
Ojo que se sigue mostrando 'posted' como tipo object

In [ ]:
df_user_reviews['posted'] = pd.to_datetime(df_user_reviews['posted'], format='%B %d %Y').dt.strftime('%Y-%m-%d')
print(df_user_reviews['posted'].dtype)

object


Pintamos una muestra aleatoria de 'posted'

In [ ]:
muestra = df_user_reviews.sample(n=100, random_state= 50)['posted'].head(50) 
matriz = np.array(muestra).reshape(10, 5) # Convierte la serie 'posted_muestra' en un arreglo 15x7
print(matriz)

[['2013-11-28' '2015-04-05' '2014-05-14' '2014-01-17' '2010-12-18']
 ['2014-01-03' '2012-12-16' '2015-04-05' '2015-10-30' '2014-03-15']
 ['2015-07-23' '2015-10-08' '2015-06-20' '2012-07-13' '2015-01-24']
 ['2012-09-05' '2015-03-20' '2014-11-23' '2014-03-15' '2015-08-21']
 ['2014-12-11' '2014-07-19' '2014-05-27' '2015-01-12' '2014-01-09']
 ['2014-04-17' '2015-04-08' '2015-07-19' '2013-12-19' '2015-10-26']
 ['2013-12-08' '2015-07-10' '2014-08-29' '2014-06-25' '2014-05-20']
 ['2014-02-08' '2014-01-22' '2013-12-01' '2015-06-07' '2014-06-04']
 ['2015-10-06' '2014-03-18' '2015-11-01' '2014-09-24' '2015-04-02']
 ['2015-12-05' '2014-01-12' '2015-05-03' '2015-01-18' '2013-05-25']]


## Tratamiento del campo 'recommend'

### Exploramos 'recommend'

In [ ]:
print('columnas en df_user_reviews: ','[' + ', '.join(df_user_reviews.keys()) + ']')


columnas en df_user_reviews:  [user_id, user_url, funny, posted, last_edited, item_id, helpful, recommend, review]


In [ ]:
print(df_user_reviews.shape)
print("Tipo de dato en 'recommend':",df_user_reviews['recommend'].dtype)
df_user_reviews.head(3)

(48498, 9)
Tipo de dato en 'recommend': bool


,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-11-05,,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-07-15,,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-04-21,,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...


Buscamos todos los valores unicos en 'recommend'

In [ ]:
unique_values = df_user_reviews['recommend'].unique() #muestra valores únicos en la columna 'price'
print("Número de valores únicos en 'recommend': ", len(unique_values))
print(unique_values)

Número de valores únicos en 'recommend':  2
[ True False]


## Tratamiento del campo 'review'

### Exploramos 'review'

In [ ]:
print('columnas en df_user_reviews: ','[' + ', '.join(df_user_reviews.keys()) + ']')


columnas en df_user_reviews:  [user_id, user_url, funny, posted, last_edited, item_id, helpful, recommend, review]


In [ ]:
print(df_user_reviews.shape)
print("Tipo de dato en 'review':",df_user_reviews['review'].dtype)
df_user_reviews.head(3)

(48498, 9)
Tipo de dato en 'review': object


,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-11-05,,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-07-15,,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-04-21,,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...


Checkamos si hay reseñas que no son cadenas de texto en 'review'  

In [ ]:

# Filtra las reseñas que no son cadenas de texto y almacénalas en una lista
reseñas_no_texto = df_user_reviews[~df_user_reviews['review'].apply(lambda x: isinstance(x, str))]['review'].tolist()

print(df_user_reviews.shape)
print('Cantidad de reseñas invalidas:',len(reseñas_no_texto))
# Imprime la lista de reseñas que no son cadenas de texto
print(reseñas_no_texto)

#df_user_reviews_prepared.head(50)

(48498, 9)
Cantidad de reseñas invalidas: 0
[]


### Transformamos 'review' en 'sentiment_analysis'

In [ ]:
# Crear una instancia del analizador de sentimientos
sia = SentimentIntensityAnalyzer()

# Función para realizar el análisis de sentimiento y mapear los resultados a la escala deseada
def map_sentiment(score):
    if score >= 0.05:
        return 2  # Positivo
    elif score <= -0.05:
        return 0  # Malo
    else:
        return 1  # Neutral

# Aplicar análisis de sentimiento y mapear los resultados a la escala
df_user_reviews['sentiment_scores'] = df_user_reviews['review'].apply(lambda x: sia.polarity_scores(x)['compound'])
df_user_reviews['sentiment_analysis'] = df_user_reviews['sentiment_scores'].apply(map_sentiment)

# Eliminar la columna 'sentiment_scores' si no se necesita
df_user_reviews.drop(columns=['sentiment_scores'], inplace=True)

# Si no hay reseña escrita, establecer 'sentiment_analysis' en 1
df_user_reviews['sentiment_analysis'].fillna(1, inplace=True)

# Opcional: Reemplazar el campo 'review' con 'sentiment_analysis' si es necesario
df_user_reviews.drop(columns=['review'], inplace=True)
# df_user_reviews_prepared_prueba.rename(columns={'sentiment_analysis': 'review'}, inplace=True)

0        0.8481
1        0.2263
2        0.9117
3        0.9566
4        0.9708
          ...  
58378    0.0000
58381    0.7650
58391   -0.4939
58393    0.6739
58402    0.7089
Name: sentiment_scores, Length: 48498, dtype: float64


 Verificamos los cambios

In [ ]:
print("Tipo de dato en 'sentiment_analysis':",df_user_reviews['sentiment_analysis'].dtype)
print("Cantidad de velores faltantes en 'sentiment_analysis'",df_user_reviews['sentiment_analysis'].isnull().sum())
df_user_reviews.head(3)

Tipo de dato en 'sentiment_analysis': int64
Cantidad de velores faltantes en 'sentiment_analysis' 0


,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-11-05,,1250,No ratings yet,True,2
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-07-15,,22200,No ratings yet,True,2
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-04-21,,43110,No ratings yet,True,2


## Guardamos el dataset preparado

In [ ]:
#df_user_reviews.to_csv('../Data/df_user_reviews_prepared.csv', index=False)

# CREACION DE LOS ENPOINT DATASETS

Terminada la preparacion de los datos en 'df_user_reviews' toca crear los diferentes datasets en formato parquet que van alimentar al servidor de consultas

## Endpoint `'def userdata(user_id:str)'`

### Transformacion

Filtramos los campos 'id' y 'price'. Eliminamos el resto

In [ ]:
columnas_a_mantener = ['user_id', 'recommend']
df_user_reviews_userdata_user_id_respuesta2 = df_user_reviews[columnas_a_mantener]
print(df_user_reviews_userdata_user_id_respuesta2.shape)
print(df_user_reviews_userdata_user_id_respuesta2.dtypes)
df_user_reviews_userdata_user_id_respuesta2.head(3)

(48498, 2)


,user_id,recommend
0,76561197970982479,True
1,76561197970982479,True
2,76561197970982479,True


### To Parquet

In [ ]:
df_user_reviews_userdata_user_id_respuesta2.to_parquet('../Data/df_user_reviews_userdata_user_id_respuesta2.parquet', index=False)

## Endpoint `'def best_developer_year(año : int)'`

### Transformacion

Filtramos los campos 'item_id', 'posted', 'recommend' y 'sentiment_analysis'. Eliminamos el resto

In [ ]:
columnas_a_mantener = ['item_id','posted','recommend', 'sentiment_analysis']
df_user_reviews_best_developer_year_año = df_user_reviews[columnas_a_mantener]

print(df_user_reviews_best_developer_year_año.shape)
print(df_user_reviews_best_developer_year_año.dtypes)
df_user_reviews_best_developer_year_año.head(3)

(48498, 4)
item_id               object
posted                object
recommend               bool
sentiment_analysis     int64
dtype: object


,item_id,posted,recommend,sentiment_analysis
0,1250,2011-11-05,True,2
1,22200,2011-07-15,True,2
2,43110,2011-04-21,True,2


### To Parquet

In [ ]:
df_user_reviews_best_developer_year_año.to_parquet('../Data/df_user_reviews_best_developer_year_año.parquet', index=False)

## Endpoint `'def recomendacion_usuario(user_id: srt)'`

A continuacion preparamos el dataset que alimentará a la etapa de preprocesamiento de datos del sistema de recomendacion

### Transformacion

Filtramos los campos 'user_id','item_id', 'recommend' y 'sentiment_analysis'. Eliminamos el resto

In [ ]:
columnas_a_mantener = ['user_id','item_id','recommend', 'sentiment_analysis']
df_user_reviews_recommendacion_usuario_user_id = df_user_reviews[columnas_a_mantener]

print(df_user_reviews_recommendacion_usuario_user_id.shape)
print(df_user_reviews_recommendacion_usuario_user_id.dtypes)
print('Datos faltantes:')
print(df_user_reviews_recommendacion_usuario_user_id.isnull().sum())
df_user_reviews_recommendacion_usuario_user_id.head(3)

(48498, 4)
user_id               object
item_id               object
recommend               bool
sentiment_analysis     int64
dtype: object
Datos faltantes:
user_id               0
item_id               0
recommend             0
sentiment_analysis    0
dtype: int64


,user_id,item_id,recommend,sentiment_analysis
0,76561197970982479,1250,True,2
1,76561197970982479,22200,True,2
2,76561197970982479,43110,True,2


### To Parquet

In [ ]:
df_user_reviews_recommendacion_usuario_user_id.to_parquet('../Data/df_user_reviews_recommendacion_usuario_user_id.parquet', index=False)

# Codigo no relacionado al proyecto

## Listar variables en memoria 

In [ ]:
import builtins
import sys

# Obtener la lista de nombres de variables
variable_names = dir()
variables_descartables = ['exit','quit','Out','In','sys','var','var_name','variable_names','builtins','contenido','size','user_defined_variables','variables_descartables']
# Filtrar las variables creadas manualmente
user_defined_variables = [
    var for var in variable_names 
    #if not var.startswith('_')
    if var not in dir(builtins) and not var.startswith('_') and var not in variables_descartables
]
#print(user_defined_variables)
print('Variables encontradas: ', len(user_defined_variables))
# Imprimir el tamaño y la fecha de creación de las variables
for var_name in user_defined_variables:
    var = globals()[var_name]
    if sys.getsizeof(var) >= 100000:
        size = sys.getsizeof(var)  
        contenido = eval(var_name)
        print(f"La variable {var_name} pesa {size} bytes y es de tipo {str(type(contenido))[8:-2]}")

Variables encontradas:  26
La variable df_user_items pesa 71310066 bytes y es de tipo pandas.core.frame.DataFrame
La variable df_user_items_userdata_prepared_desanidado pesa 2088152303 bytes y es de tipo pandas.core.frame.DataFrame
La variable filas_repetidas1 pesa 1438635 bytes y es de tipo pandas.core.frame.DataFrame
La variable ides_unicos pesa 567408 bytes y es de tipo numpy.ndarray
